In [95]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [96]:
df_drivers = pd.read_csv('driver_ids.csv', parse_dates=['driver_onboard_date'])
df_rides = pd.read_csv('ride_ids.csv')
df_rtsp = pd.read_csv('ride_timestamps_pvt.csv', parse_dates=['accepted_at', 'arrived_at', 'dropped_off_at', 'picked_up_at', 'requested_at'])
df_r = pd.read_csv('rides_revenue.csv', parse_dates=['driver_onboard_date', 'dropped_off_at'])

### Average Projected Lifetime Of A Driver (Months)

##### Using Average Months Active

In [97]:
## Assumption: average projected lifetime of a driver = last ride done - onboard date. If no rides complete, then lifetime is 0 months
# Find the last ride that a driver did by looking at their max drop off date. 
max_drop_off_time = df_r.groupby(['driver_id'])['dropped_off_at'].max()

# Merge (left join) that with when drivers were onboarded. Looks like 100 (~9%) drivers never completed a ride
df = pd.merge(df_drivers, max_drop_off_time, how='left', on=['driver_id'])

# Get month delta between last ride and onboard date. Fill in your no-ride drivers with 0 months
df['time_to_last_ride'] = df['dropped_off_at'] - df['driver_onboard_date']
df['time_to_last_ride_m'] = df['time_to_last_ride'] / np.timedelta64(1, 'M')
df['time_to_last_ride_m'].fillna(0, inplace=True)

# Calc the mean number of months
average_driver_lifetime = df['time_to_last_ride_m'].mean()

print ("The Average Lifetime Of A Driver (using average months active) Is: {} Months".format(average_driver_lifetime))

The Average Lifetime Of A Driver (using average months active) Is: 1.635720757643673 Months


##### Churn Rate
We are going to grab a **simple weekly churn rate**. Of all the drivers that were active a given week, how many were active next week? Then we will take the reciprocal to see how many weeks they're expected to be active. This doesn't account for drivers who never took a ride.

In [145]:
# Create a simple table that has all rides and their drop off date. We are assuming the ride date is drop off date
dfw = df_r[['driver_id', 'ride_id', 'dropped_off_at']]

In [146]:
min_ride_date = dfw['dropped_off_at'].min()
max_ride_date = dfw['dropped_off_at'].max()
date_range = pd.date_range(min_ride_date, max_ride_date)

In [155]:
date_window = 28 # 28 for 28 days in a month
churn_rates = []

# Go through each date you have and then calc a rolling churn metric.
# We are making sure we don't go all the way to the end of the list
for date in date_range[:(-60)]:
    
    # Get the dates of your windows
    active_close = date + pd.Timedelta(date_window, unit='D')
    churn_close = date + pd.Timedelta(date_window * 2, unit='D')
    
    # Get a list of drivers who were active between the date your on and then end of your active period
    c1 = dfw['dropped_off_at'] >= date
    c2 = dfw['dropped_off_at'] <= active_close
    active_drivers = dfw[c1 & c2]['driver_id'].unique()
    
    # Get a list of drivers who were active between the end of your active period and your churn window
    c3 = dfw['dropped_off_at'] >= active_close
    c4 = dfw['dropped_off_at'] <= churn_close
    active_drivers_churn_window = dfw[c3 & c4]['driver_id'].unique()
    
    # See what the overlap is. This is how many drivers were still active after your churn window
    still_active_drivers = set(active_drivers) & set(active_drivers_churn_window)
    
    churn_rate = 1 - (len(still_active_drivers) / len(active_drivers))
    
    # Record your churn rates
    churn_rates.append(churn_rate)
    
#     print (date, active_close, churn_close, len(still_active_drivers), len(active_drivers), churn_rate)
    
average_churn_rate = np.mean(churn_rates)
average_driver_lifetime = 1 / average_churn_rate

print ("Average Churn Rate: {:.3f}".format(average_churn_rate))
print ("Expected Months Active (1 / churn rate): {:,.2f} Months".format(average_driver_lifetime))

Average Churn Rate: 0.108
Expected Months Active (1 / churn rate): 9.29 Months


### Average Revenue Per Driver Per Month

1. Total up total Revenue Per Driver
2. Use the length of time they've been driving (from above calc)
3. Divide the two metrics to get your average revenue per driver per month

In [156]:
# Find out how much $$ a driver has made over their lifetime
df_driver_total_revenue = df_r.groupby(['driver_id'])['ride_revenue'].sum()

In [157]:
# Drop drivers who haven't completed a ride
df = df[-df['dropped_off_at'].isna()]

In [158]:
# Merge with your dataset that has the length of time a driver has been driving
df_driver_monthly_revenue = pd.merge(df, df_driver_total_revenue, how='left', on=['driver_id'])
df_driver_monthly_revenue = df_driver_monthly_revenue[['driver_id', 'time_to_last_ride_m', 'ride_revenue']]

# Divide their total revenue by the months they've been driving to get revenue per month. Fill the NaNs with 0
df_driver_monthly_revenue['driver_revenue_per_month'] = df_driver_monthly_revenue['ride_revenue'] / df_driver_monthly_revenue['time_to_last_ride_m']
df_driver_monthly_revenue.fillna(0, inplace=True)

In [159]:
# Get the mean revenue per driver per month from the dataset you just made
average_driver_revenue_per_month = df_driver_monthly_revenue['driver_revenue_per_month'].mean()
print ("The average driver revenue per month is: ${:,.2f}".format(average_driver_revenue_per_month))

The average driver revenue per month is: $1,635.22


### Average Lifetime Value Of A Driver

Average Length Of Driver Lifetime (months) * Average Revenue Per Driver Per Month

In [160]:
print ("The Average Lifetime Of A Driver Is: {:,.2f} Months".format(average_driver_lifetime))
print ("The Average Driver Revenue Per Month is: ${:,.2f}".format(average_driver_revenue_per_month))
print ("")

drive_lifetime_value = average_driver_lifetime * average_driver_revenue_per_month
print ("The Driver Lifetime Value is: ${:,.2f}".format(drive_lifetime_value))

The Average Lifetime Of A Driver Is: 9.29 Months
The Average Driver Revenue Per Month is: $1,635.22

The Driver Lifetime Value is: $15,189.75


In [162]:
# To get the actual buckets for your histogram. This will be used to make the charts on gSheets

# df_driver_monthly_revenue['driver_revenue_per_month'].hist(bins=30)
# _hist = np.histogram(df_driver_monthly_revenue['driver_revenue_per_month'], bins=30, range=(0,7500))
# pd.DataFrame(zip( _hist[1], _hist[0])).to_clipboard()